## 2 Forest Fire Sampling IV

#### Building the model

Models consist of __agents__ and the __environment__
In the forest fire model, the agent is a tree and it's state can be
- fine(unburned)
- on fire
- burnt


The environment is the graph of pinterest users.


BFS for graph to spread fire


In [1]:
import numpy as np
from collections import deque
import pymongo
from pymongo import MongoClient
print( 'Mongo version', pymongo.__version__)

Mongo version 3.7.0


In [2]:
#main db
config = {
  'host': '127.0.0.1',
  'username': 'mongo_user',
  'password': 'password',
  'authSource': 'pin'
}

client = MongoClient(**config)

db = client.pin
collection1=db.users
collection2=db.sampled_users
collection3=db.more_sampled
collection4=db.sampled1
collection5=db.sampled2

In [3]:
### Forest Fire Model

In [5]:
collection5.delete_many({})

In [6]:
collection5.find_one()


In [79]:
def burn_or_not(p = 1.0):
    return (np.random.random() <=p)

def select_node():
    node = str(np.random.randint(1,50000000))
    if node not in sampled:
        return node

def spread_the_fire_bfs(obj):
    explored = set()
    sampled = set()
    collection5.delete_many({})
    
    sampled.add(obj['user_id'])
    # keep track of explored nodes
    # keep track of all the paths to be checked
    queue = [obj['user_id']]
    #print(queue)
    while queue:
        # pop first node from queue
        current_user_id = queue.pop(0).strip()
        if current_user_id not in explored:
            #add node to list of explored nodes
            explored.add(current_user_id)
            #insert obj to collection2
            object_to_add = collection1.find_one({'user_id': current_user_id})
            if object_to_add is None:
                continue
            collection5.insert_one(object_to_add)
              
            burn = burn_or_not()
            if True:
                queue.extend(object_to_add['followers'])
            
        if collection5.count_documents({}) > 1000:
            print("Early return")
            return

def start_the_fire():
    user_id = collection3.find_one({'follower_count': {'$eq': '10'}}, {'user_id': 1, '_id': 0})['user_id']
    obj = collection1.find_one({'user_id':user_id})
    return spread_the_fire_bfs(obj)

In [80]:
the_object = start_the_fire()
    

Early return


In [81]:
collection5.count_documents({})

1001

In [37]:
collection5.count_documents({})

1

In [14]:
len(sampled)

1104

In [83]:
x=collection5.find().count()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


In [84]:
x

1001

In [85]:
collection5.find_one({'follower_count': {'$eq': '10'}}, {'user_id': 1, '_id': 0})

{'user_id': '31018351'}